In [2]:
!pip install transformers gradio pymupdf #used dependencies

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00


In [3]:
from transformers import pipeline

# Generative QA pipeline
gen_qa = pipeline("text2text-generation", model="google/flan-t5-base", max_length=256)

# New context: Transformers in NLP
context = """
Transformers are a type of deep learning architecture introduced in the paper "Attention is All You Need" by Vaswani et al. in 2017.
They have revolutionized natural language processing (NLP) by enabling models to process entire sequences of text at once using self-attention mechanisms, instead of relying on sequential processing like RNNs or LSTMs.

A key innovation in transformers is the attention mechanism, which allows the model to weigh the relevance of different words in a sequence, regardless of their position. This results in better understanding of context and relationships in text.

Transformers form the foundation of models such as BERT, GPT, T5, and RoBERTa, which are pre-trained on large corpora and then fine-tuned for specific tasks like sentiment analysis, text classification, translation, and question answering.

Because of their scalability and performance, transformers have also been adopted in fields beyond NLP, such as computer vision, protein folding, and generative modeling.
"""

# New questions
questions = [
    "What is a transformer in NLP?",
    "Who introduced the transformer model and when?",
    "What is the attention mechanism in transformers?",
    "Which models are based on the transformer architecture?",
    "Are transformers used outside of NLP?"
]

# Ask each question
for question in questions:
    prompt = f"Answer the question based on the context:\nContext: {context}\nQuestion: {question}"
    answer = gen_qa(prompt)[0]['generated_text']
    print(f"Q: {question}")
    print(f"A: {answer}\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Q: What is a transformer in NLP?
A: enabling models to process entire sequences of text at once using self-attention mechanisms, instead of relying on sequential processing like RNNs or LSTMs

Q: Who introduced the transformer model and when?
A: Vaswani et al.

Q: What is the attention mechanism in transformers?
A: allows the model to weigh the relevance of different words in a sequence, regardless of their position

Q: Which models are based on the transformer architecture?
A: BERT, GPT, T5, and RoBERTa

Q: Are transformers used outside of NLP?
A: transformers have also been adopted in fields beyond NLP



In [7]:
import gradio as gr
import fitz  # PyMuPDF
import re
from transformers import pipeline

# Load QA pipeline once
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Clean text extracted from PDFs
def clean_text(text):
    text = text.replace('\n', ' ')  # Flatten line breaks
    text = re.sub(r'\s+', ' ', text)  # Normalize spaces
    return text.strip()

# Extract and clean text from PDF or TXT
def extract_text(file):
    if file is None:
        return ""
    fname = file.name
    if fname.endswith(".pdf"):
        try:
            doc = fitz.open(fname)
            text = ""
            for page in doc:
                text += page.get_text()
            return clean_text(text)
        except Exception as e:
            return f"⚠️ PDF extraction failed: {str(e)}"
    elif fname.endswith(".txt"):
        try:
            with open(fname, "r", encoding="utf-8") as f:
                return clean_text(f.read())
        except Exception as e:
            return f"⚠️ TXT read failed: {str(e)}"
    else:
        return "⚠️ Unsupported file type. Please upload PDF or TXT."

# Answer questions based on uploaded file
def answer_file_question(file, question):
    if file is None or not question.strip():
        return "⚠️ Please upload a file and enter a question."
    context_text = extract_text(file)
    if context_text.startswith("⚠️"):
        return context_text
    try:
        result = qa(question=question, context=context_text)
        return f"### Answer:\n{result['answer']}\n\n*Confidence: {result['score']:.2f}*"
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Answer questions based on pasted context
def answer_context_question(context, question):
    if not context.strip() or not question.strip():
        return "⚠️ Please enter context and a question."
    try:
        cleaned_context = clean_text(context)
        result = qa(question=question, context=cleaned_context)
        return f"### Answer:\n{result['answer']}\n\n*Confidence: {result['score']:.2f}*"
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# Gradio interface
with gr.Blocks(theme=gr.themes.Default()) as demo:
    gr.HTML("<script>document.title = 'DocDigest';</script>")
    gr.Markdown("<h2 style='text-align:center; color:#4A90E2;'>📚 DocDigest - The Context Summarizer</h2>")

    gr.Markdown("Upload a PDF or TXT file (optional)")
    file_input = gr.File(label="Upload PDF or TXT file", file_types=['.pdf', '.txt'], elem_id="file_input")
    file_question_input = gr.Textbox(label="Ask a question related to the uploaded file", placeholder="Type question about the file here...", lines=2)
    file_submit_btn = gr.Button("Get Answer for File")
    file_answer_output = gr.Markdown()

    gr.Markdown("Paste context text (if no file uploaded)")
    context_input = gr.Textbox(label="Paste context text here", placeholder="Paste paragraph(s) here...", lines=8)
    context_question_input = gr.Textbox(label="Ask a question related to the context", placeholder="Type question about the context here...", lines=2)
    context_submit_btn = gr.Button("Get Answer for Context")
    context_answer_output = gr.Markdown()

    file_submit_btn.click(fn=answer_file_question, inputs=[file_input, file_question_input], outputs=file_answer_output)
    context_submit_btn.click(fn=answer_context_question, inputs=[context_input, context_question_input], outputs=context_answer_output)

    gr.HTML(
        """
        <style>
        body {
            background-color: #f0f4f8;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        #file_input input[type="file"] {
            width: 100% !important;
            padding: 8px;
            border-radius: 6px;
            border: 1px solid #ccc;
            box-sizing: border-box;
        }
        #file_question_input, #context_input, #context_question_input {
            width: 100% !important;
            font-size: 14px !important;
            border-radius: 6px !important;
            border: 1px solid #ccc !important;
            padding: 8px !important;
            box-sizing: border-box;
        }
        #file_submit_btn button, #context_submit_btn button {
            background-color: #4A90E2 !important;
            color: white !important;
            font-weight: 600 !important;
            font-size: 16px !important;
            border-radius: 6px !important;
            padding: 10px 20px !important;
            margin-top: 10px !important;
            width: 100% !important;
            border: none !important;
            cursor: pointer !important;
            transition: background-color 0.3s ease;
        }
        #file_submit_btn button:hover, #context_submit_btn button:hover {
            background-color: #357ABD !important;
        }
        #file_answer_output, #context_answer_output {
            background-color: #000000 !important;
            padding: 15px !important;
            border-radius: 6px !important;
            min-height: 120px !important;
            font-size: 15px !important;
            color: #333 !important;
            white-space: pre-wrap !important;
            border: 1px solid #ddd !important;
            margin-top: 10px !important;
        }
        </style>
        """
    )

demo.launch(share=True)


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://19e277afa742702bc5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
